In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install scikit-surprise

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import pandas as pd

from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
train_fimename = '/content/gdrive/MyDrive/2021_인공지능통신/movie_train.csv'
test_filename = '/content/gdrive/MyDrive/2021_인공지능통신/movie_test_label.csv'
train_1 = pd.read_csv(train_fimename)
test_1 = pd.read_csv(test_filename)
train_data = train_1.values
test_data = test_1.values

In [ ]:
#train_data 정렬
train_sort = train_1.sort_values(by = ['movieId','userId'], ascending = True)
train_final = train_sort.drop('Unnamed: 0', axis = 1)
train_final2= train_final.reset_index(drop = True)
train1 = np.array(train_final2)

In [ ]:
#test_data 정렬
test_sort = test_1.sort_values(by = ['movieId','userId'], ascending = True)
test_final = test_sort.drop('Unnamed: 0', axis = 1)
test_final2= test_final.reset_index(drop = True)
test_set = test_final2.values
test1 = np.array(test_final2)

In [ ]:
test_final2

,userId,movieId,rating
0,7,1,3.0
1,70,1,5.0
2,75,1,3.0
3,79,1,2.0
4,15,2,2.0
...,...,...,...
1299,95,1995,3.0
1300,57,1996,2.0
1301,15,1997,5.0
1302,48,1997,3.5


In [ ]:
train_1['userId'].value_counts().sort_index(ascending = True)

1      11
2      63
3      19
4      80
5      35
       ..
96     29
97     50
98     16
99     90
100    20
Name: userId, Length: 100, dtype: int64

In [ ]:
train_1['movieId'].value_counts().sort_index(ascending = True)

1       34
2       15
3        7
4        1
5        6
        ..
1987     1
1991     1
1994     8
1995     2
1997    11
Name: movieId, Length: 970, dtype: int64

In [ ]:
user = np.zeros((5216, 100))
movie = np.zeros((5216, 970))

In [ ]:
i = 0
while i < len(train1):
  j = int(train1[i, 0]) - 1
  user[i, j] = 1
  i += 1

In [ ]:
i = 0
j = 0
while i < len(train1) - 1:
  movie[i, j] = 1
  if train1[i, 1] != train1[i + 1, 1]:
    i += 1
    j += 1
  elif i == len(train1) - 2:
    movie[i + 1, j] = 1
    break
  else:
    i += 1

In [ ]:
td1 = train_final2.drop('userId', axis = 1)
td2 = td1.drop('movieId', axis = 1)
rate = td2.reset_index(drop = True)
rating = np.array(rate)

i = 0
while i < len(train1):
  rating[i, 0] = rating[i, 0] - train_1['rating'].mean()
  i += 1

rating_list = np.array(rating).flatten().tolist()

In [ ]:
m = 5216
n = 1070
np.random.seed(1)
A = np.hstack((user, movie))
b = rating_list
x = cp.Variable(n)
cost = cp.sum_squares(A @ x - b)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

print("\nThe optimal value is", prob.value)
print("The optimal x is")
print(x.value)
print("The norm of the residual is ", cp.norm(A @ x - b, p = 2).value)


The optimal value is 3026.423821319599
The optimal x is
[-1.19858628 -0.19710866 -0.42605306 ...  0.37807971 -1.23374812
  0.47946865]
The norm of the residual is  55.0129423074207


In [ ]:
train_copy = train1[:]
test_copy = test1[:]

In [ ]:
diff1 = train_final2.drop('userId', axis = 1)
diff1 = diff1.drop('rating', axis = 1)
diff1 = diff1.reset_index(drop = True)
diff1_movie = np.array(diff1)

diff2 = train_final2.drop('movieId', axis = 1)
diff2 = diff2.drop('rating', axis = 1)
diff2 = diff2.reset_index(drop = True)
diff2_user = np.array(diff2)

diff3 = test_final2.drop('userId', axis = 1)
diff3 = diff3.drop('rating', axis = 1)
diff3 = diff3.reset_index(drop = True)
diff3_movie = np.array(diff3)

diff4 = test_final2.drop('movieId', axis = 1)
diff4 = diff4.drop('rating', axis = 1)
diff4 = diff4.reset_index(drop = True)
diff4_user = np.array(diff4)

In [ ]:
temp3 = []
for i in diff3_movie:
    if i not in diff1_movie:
        temp3.append(i)

In [ ]:
i = 0
k = 1
while i < len(train1) - 1:
  j = int(train1[i, 0])
  if train1[i, 1] == train1[i + 1, 1]:
    train_copy[i, 2] = train_1['rating'].mean() + x.value[j - 1] + x.value[99 + k]
    i += 1
  else:
    k += 1
    train_copy[i, 2] = train_1['rating'].mean() + x.value[j - 1] + x.value[99 + k]
    i += 1

train_copy[len(train1) - 1, 2] = train_1['rating'].mean() + x.value[77] + x.value[1069]

i = 0
j = 0
while i < len(test1):
  test_userid = int(test1[i, 0])
  train_userid = int(train1[j, 0])
  if test_userid == train_userid:
    test_copy[i, 2] = train_1['rating'].mean() + x.value[test_userid - 1]
    i += 1
    j = 0
  else:
    j += 1

i = 0
j = 0
k = 1
while i < len(test1):
  test_movieid = int(test1[i, 1])
  train_movieid = int(train1[j, 1])
  if test_movieid == train_movieid:
    test_copy[i, 2] = test_copy[i, 2] + x.value[99 + k]
    i += 1
    j = 0
    k = 1
  elif test_movieid in temp3:
    i += 1
    j = 0
    k = 1
  else:
    if train1[j, 1] == train1[j + 1, 1]:
      j += 1
    else:
      j += 1
      k += 1

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

test_rating = test_final2.drop('userId', axis = 1)
test_rating = test_rating.drop('movieId', axis = 1)
test_rating = test_rating.reset_index(drop = True)
rating_t = np.array(test_rating)

baseline_rating = np.delete(test_copy, 1, axis = 1)
rating_b = np.delete(baseline_rating, 0, axis = 1)
rmse = sqrt(mean_squared_error(rating_t, rating_b))

In [ ]:
rmse

0.9582019278785457

In [ ]:
data = pd.DataFrame(train_copy)

data.to_csv('/content/gdrive/MyDrive/2021_인공지능통신/rating.csv', index=False, header=False)

In [ ]:
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0, 5))

data_set = Dataset.load_from_file('/content/gdrive/MyDrive/2021_인공지능통신/rating.csv',reader=reader)

In [ ]:
train_set, valid_set = train_test_split(data_set, test_size=0.1, shuffle=True)

In [ ]:
sim_options = {'name': 'cosine','user_based': False}
model = KNNBasic(k = 10, sim_options = sim_options, verbose=2)
model.fit(train_set)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
prediction = model.test(test_set)

accuracy.rmse(prediction)

RMSE: 1.0438


1.0438242584680162